<a href="https://colab.research.google.com/github/groda/big_data/blob/master/Hadoop_single_node_cluster_setup_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hadoop single-node cluster setup with Python

## Constants

In [1]:
# URL for downloading Hadoop
HADOOP_URL = "https://dlcdn.apache.org/hadoop/common/stable/hadoop-3.3.4.tar.gz"

# logging level (should be one of: DEBUG, INFO, WARNING, ERROR, CRITICAL) 
LOGGING_LEVEL = "DEBUG" #@param ["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"]

# JAVA PATH
JAVA_PATH = '/usr/lib/jvm/java-11-openjdk-amd64'

## Setup logging

In [2]:
import logging

for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging_level = getattr(logging, LOGGING_LEVEL.upper(), 10)

logging.basicConfig(level=logging_level, \
                    format='%(asctime)s - %(levelname)s: %(message)s', \
                    datefmt='%d-%b-%y %I:%M:%S %p')

logger = logging.getLogger('my_logger')

## Start `sshd` server

### Install `openssh-server`

In [3]:
import subprocess

In [4]:
logger.info("Installing {}".format("openssh-server"))
cmd = ["apt-get", "install", "openssh-server"]
result = subprocess.check_output(cmd, stderr=subprocess.STDOUT)


27-Oct-22 09:44:04 PM - INFO: Installing openssh-server


### Configure `sshd` service

In [5]:
import os

ssh_config_file = "/etc/ssh/ssh_config"
with open(ssh_config_file, "r+") as f:
    var = 'StrictHostKeyChecking no'
    line_found = any(line.strip().startswith(var) for line in f)
    if not line_found:
      f.seek(0, os.SEEK_END)
      f.write(var +"\n")

### Start `sshd` service

In [6]:
logger.info("Starting {}".format("openssh-server"))
cmd = ["/etc/init.d/ssh", "restart"]
result = subprocess.check_output(cmd, stderr=subprocess.STDOUT)

27-Oct-22 09:44:06 PM - INFO: Starting openssh-server


### Create private/public key 

The private/public key pair is needed for passwordless authentication.

Note: for some reason `Crypto` doesn't work without `pycryptodome` (see [ModuleNotFoundError: No module named 'Crypto' Error](https://stackoverflow.com/questions/51824628/modulenotfounderror-no-module-named-crypto-error).

In [7]:
%%bash
pip install Crypto
pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
import Crypto

In [9]:
from Crypto.PublicKey import RSA
key = RSA.generate(2048)
if not os.path.exists(os.path.join(os.path.expanduser('~'), '.ssh')):
  os.makedirs(os.path.join(os.path.expanduser('~'), '.ssh'), mode=700)
#os.chmod(os.path.join(os.path.expanduser('~'),'.ssh'), int('700', base=8))

with open(os.path.join(os.path.expanduser('~'), '.ssh/id_rsa'), 'wb') as f:
  os.chmod(os.path.join(os.path.expanduser('~'),'.ssh/id_rsa'), int('0600', base=8))
  f.write(key.export_key('PEM', passphrase=None))
pubkey = key.publickey()

with open(os.path.join(os.path.expanduser('~'), '.ssh/id_rsa.pub'), 'wb') as f:
    f.write(pubkey.exportKey('OpenSSH'))

In [10]:
with open(os.path.join(os.path.expanduser('~'), '.ssh/authorized_keys'), 'w') as a:
  with open(os.path.join(os.path.expanduser('~'),'.ssh/id_rsa.pub'), 'r') as f:
    a.write(f.read())
  os.chmod(os.path.join(os.path.expanduser('~'), '.ssh/authorized_keys'), int('600', base=8))
  

#### Test `ssh` connection

The output should be 'hi!' if the passphraseless connection is working.

In [11]:
!ssh localhost "echo hi!"

hi!


## Setup Hadoop 


### Download core Hadoop
Download the latest stable version of the core Hadoop distribution from one of the download mirrors locations https://www.apache.org/dyn/closer.cgi/hadoop/common/.

In [12]:
import urllib.request
import os

file_name = os.path.basename(HADOOP_URL)
if os.path.isfile(file_name):
   logger.info("{} already exists, not downloading".format(file_name))
else:
  logger.info("Downloading {}".format(file_name))
  urllib.request.urlretrieve(HADOOP_URL, file_name)

27-Oct-22 09:44:18 PM - INFO: hadoop-3.3.4.tar.gz already exists, not downloading


### Uncompress archive

In [13]:
import tarfile

dir_name = file_name[:-7]
if os.path.exists(dir_name):
  logger.info("{} is already uncompressed".format(file_name))
else:
  logger.info("Uncompressing {}".format(file_name))
  tar = tarfile.open(file_name)
  tar.extractall()
  tar.close()

27-Oct-22 09:44:18 PM - INFO: hadoop-3.3.4.tar.gz is already uncompressed


### Configure Hadoop



##### Environment variables

In [14]:
os.environ['HADOOP_HOME'] = os.path.join('/content', dir_name)
os.environ['PATH'] = ':'.join([os.path.join(os.environ['HADOOP_HOME'], 'bin'), os.environ['PATH']])

##### Hadoop configuration files

We're going to edit two important Hadoop configuration files:


*   `core-site.xml` containing the *site-specific configuration* of the Hadoop core
*   `hdfs-site.xml` containing the *site-specific configuration* of Hadoop's HDFS

The file `hadoop-env.sh` is a helper file containing environment variables needed by the shell scripts used to start the cluster.

All these files can be found in `/etc/hadoop` under `$HADOOP_HOME` (the Hadoop installation folder).

In [15]:
core_site_file = os.path.join(os.environ['HADOOP_HOME'],'etc/hadoop/core-site.xml')
hdfs_site_file = os.path.join(os.environ['HADOOP_HOME'],'etc/hadoop/hdfs-site.xml')
hadoop_env_file = os.path.join(os.environ['HADOOP_HOME'],'etc/hadoop/hadoop-env.sh')

In [16]:
import xml.etree.ElementTree as ET
import xml.dom.minidom

In [17]:
def edit_conf_file(file, propertyname, propertyvalue):
  tree = ET.parse(file)
  root = tree.getroot()

  # remove all properties
  for el in list(root):
      root.remove(el)

  logger.info("add property {} to {}".format(propertyname, file))
  property = ET.SubElement(root, 'property')
  name = ET.SubElement(property, 'name')
  name.text = propertyname
  value = ET.SubElement(property, 'value')
  value.text = propertyvalue
  tree.write(file, encoding='utf-8', xml_declaration=True)

  # pretty-print
  dom = xml.dom.minidom.parse(file) 
  print(dom.toprettyxml())

edit_conf_file(core_site_file, 'fs.defaultFS', 'hdfs://localhost:9000')
edit_conf_file(hdfs_site_file, 'dfs.replication', '1')

27-Oct-22 09:44:18 PM - INFO: add property fs.defaultFS to /content/hadoop-3.3.4/etc/hadoop/core-site.xml
27-Oct-22 09:44:18 PM - INFO: add property dfs.replication to /content/hadoop-3.3.4/etc/hadoop/hdfs-site.xml


<?xml version="1.0" ?>
<configuration>
	

	<property>
		<name>fs.defaultFS</name>
		<value>hdfs://localhost:9000</value>
	</property>
</configuration>

<?xml version="1.0" ?>
<configuration>
	


	<property>
		<name>dfs.replication</name>
		<value>1</value>
	</property>
</configuration>



In [18]:
users = ['HDFS_NAMENODE_USER', \
         'HDFS_DATANODE_USER', \
         'HDFS_SECONDARYNAMENODE_USER', \
         'YARN_RESOURCEMANAGER_USER', \
         'YARN_NODEMANAGER_USER']

logger.info("Editing {}".format(hadoop_env_file))

with open(hadoop_env_file, "r+") as f:
  for u in users:
    var = 'export ' + u + '=root'
    line_found = any(line.startswith(var) for line in f)
    if not line_found:
      f.seek(0, os.SEEK_END)
      f.write(var +"\n")
  line_found = any(line.startswith('export JAVA_HOME=') for line in f)
  if not line_found:
      f.seek(0, os.SEEK_END)
      f.write('export JAVA_HOME='+JAVA_PATH+"\n")

27-Oct-22 09:44:18 PM - INFO: Editing /content/hadoop-3.3.4/etc/hadoop/hadoop-env.sh


##### A note on Hadoop's default configuration files

Hadoop's default configuration files are:

*  `core-default.xml`
*  `hdfs-default.xml`
*  `yarn-default.xml` 
*  `mapred-default.xml`

They are read-only files that contain all default values for Hadoop properties (see [cluster setup](https://hadoop.apache.org/docs/stable/hadoop-project-dist/hadoop-common/ClusterSetup.html#Configuring_Hadoop_in_Non-Secure_Mode)) and can be viewed at:


In [19]:
import glob
list(glob.iglob(os.environ['HADOOP_HOME']+ '/**/*default.xml', recursive=True))

['/content/hadoop-3.3.4/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs-rbf/hdfs-rbf-default.xml',
 '/content/hadoop-3.3.4/share/doc/hadoop/hadoop-project-dist/hadoop-hdfs/hdfs-default.xml',
 '/content/hadoop-3.3.4/share/doc/hadoop/hadoop-project-dist/hadoop-common/core-default.xml',
 '/content/hadoop-3.3.4/share/doc/hadoop/hadoop-yarn/hadoop-yarn-common/yarn-default.xml',
 '/content/hadoop-3.3.4/share/doc/hadoop/hadoop-mapreduce-client/hadoop-mapreduce-client-core/mapred-default.xml']

By providing site-specific configuration files one can override any of the default properties.

In our case we set `fs.defaultFS` (the URI of the default file system) to `hdfs://localhost:9000` and changed `dfs.replication` to $1$ (the default is $3$) in `hdfs-site.xml`.


### Launch a single-node Hadoop cluster

#### Initialize the namenode

In [21]:
logger.info("Formatting NameNode")
cmd = ["hdfs", "namenode", "-format", "-nonInteractive"]
result = subprocess.call(cmd, stderr=subprocess.STDOUT)

27-Oct-22 09:44:18 PM - INFO: Formatting NameNode


## Start Hadoop

In [22]:
!$HADOOP_HOME/sbin/start-dfs.sh

Starting namenodes on [localhost]
localhost: namenode is running as process 1050.  Stop it first and ensure /tmp/hadoop-root-namenode.pid file is empty before retry.
Starting datanodes
localhost: datanode is running as process 1173.  Stop it first and ensure /tmp/hadoop-root-datanode.pid file is empty before retry.
Starting secondary namenodes [3b8744d84cec]
3b8744d84cec: secondarynamenode is running as process 1400.  Stop it first and ensure /tmp/hadoop-root-secondarynamenode.pid file is empty before retry.


## Check namenode

In [26]:
!hdfs dfsadmin -report

Configured Capacity: 115658190848 (107.72 GB)
Present Capacity: 89248182272 (83.12 GB)
DFS Remaining: 89248153600 (83.12 GB)
DFS Used: 28672 (28 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (1):

Name: 127.0.0.1:9866 (localhost)
Hostname: 3b8744d84cec
Decommission Status : Normal
Configured Capacity: 115658190848 (107.72 GB)
DFS Used: 28672 (28 KB)
Non DFS Used: 26393231360 (24.58 GB)
DFS Remaining: 89248153600 (83.12 GB)
DFS Used%: 0.00%
DFS Remaining%: 77.17%
Configured Cache Capacity: 0 (0 B)
Cache

In [28]:
!npm install -g localtunnel

/tools/node/bin/lt -> /tools/node/lib/node_modules/localtunnel/bin/lt.js
+ localtunnel@2.0.2
updated 1 package in 0.839s


In [29]:
!lt --port 9870

your url is: https://shaggy-experts-wait-34-135-241-210.loca.lt
^C
